In [46]:
// loading the data into a datset 
val dataset=List((1,"2013-07-25",11599,"CLOSED"),(2,"2014-07-25",256,"PENDING"),(3,"2013-07-25",11599,"COMPLETE"),(4,"2019-07-25",8827,"CLOSED"))

dataset = List((1,2013-07-25,11599,CLOSED), (2,2014-07-25,256,PENDING), (3,2013-07-25,11599,COMPLETE), (4,2019-07-25,8827,CLOSED))


lastException: Throwable = null


List((1,2013-07-25,11599,CLOSED), (2,2014-07-25,256,PENDING), (3,2013-07-25,11599,COMPLETE), (4,2019-07-25,8827,CLOSED))

In [52]:
// converting the dataset into a rdd then convert it to a dataframe
val rdd=sc.parallelize(dataset)
// converting the rdd to a dataframe 
val df =rdd.toDF("order_id","order_date","customer_id","status")

rdd = ParallelCollectionRDD[2] at parallelize at <console>:43
df = [order_id: int, order_date: string ... 2 more fields]


[order_id: int, order_date: string ... 2 more fields]

In [53]:
// converting the list datastructure to a dataset using a Structured API and converting to a dataframe
val st_df=spark.createDataFrame(dataset).toDF("order_id","order_date","customer_id","status")


st_df = [order_id: int, order_date: string ... 2 more fields]


[order_id: int, order_date: string ... 2 more fields]

In [55]:
// checking the schema as we have to convert the string to date datatype
st_df.printSchema()
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.DateType
import spark.implicits._


Unknown Error: <console>:44: error: stable identifier required, but this.$line7$read.spark.implicits found.
       import spark.implicits._
                    ^


In [56]:
// now let us convert the order_date to epoxy time using a timestamp function
// for this we will use the .withColumn because it will replace the column or add new column

val st_df1=st_df.withColumn("order_date",unix_timestamp(col("order_date").cast(DateType)))

st_df1 = [order_id: int, order_date: bigint ... 2 more fields]


[order_id: int, order_date: bigint ... 2 more fields]

In [57]:
// checking whether the date columns has been converted or not.
st_df1.printSchema()
"""root
 |-- order_id: integer (nullable = false)
 |-- order_date: long (nullable = true)
 |-- customer_id: integer (nullable = false)
 |-- status: string (nullable = true)
"""
// as we can see that the datatype of order has been changed


root
 |-- order_id: integer (nullable = false)
 |-- order_date: long (nullable = true)
 |-- customer_id: integer (nullable = false)
 |-- status: string (nullable = true)



"root
-- order_id: integer (nullable = false)
-- order_date: long (nullable = true)
-- customer_id: integer (nullable = false)
-- status: string (nullable = true)
"


In [58]:
// cretaing a new column with name "newid" and make sure it has unique ids and using a montonic increasing function
var df2=st_df1.withColumn("newid",monotonically_increasing_id())

df2 = [order_id: int, order_date: bigint ... 3 more fields]


[order_id: int, order_date: bigint ... 3 more fields]

In [42]:
df2.show()
"""+--------+----------+-----------+--------+-----+
|order_id|order_date|customer_id|  status|newid|
+--------+----------+-----------+--------+-----+
|       1|1374724800|      11599|  CLOSED|    0|
|       2|1406260800|        256| PENDING|    1|
|       3|1374724800|      11599|COMPLETE|    2|
|       4|1564027200|       8827|  CLOSED|    3|
+--------+----------+-----------+--------+-----+"""
// checking whether the new column has been added 

+--------+----------+-----------+--------+-----+
|order_id|order_date|customer_id|  status|newid|
+--------+----------+-----------+--------+-----+
|       1|1374724800|      11599|  CLOSED|    0|
|       2|1406260800|        256| PENDING|    1|
|       3|1374724800|      11599|COMPLETE|    2|
|       4|1564027200|       8827|  CLOSED|    3|
+--------+----------+-----------+--------+-----+



+--------+----------+-----------+--------+-----+
order_id|order_date|customer_id|  status|newid|
+--------+----------+-----------+--------+-----+
       1|1374724800|      11599|  CLOSED|    0|
       2|1406260800|        256| PENDING|    1|
       3|1374724800|      11599|COMPLETE|    2|
       4|1564027200|       8827|  CLOSED|    3|
+--------+----------+-----------+--------+-----+


In [59]:
// dropping the column which have same order_date and customer_id column values
val df3=df2.dropDuplicates("order_date","customer_id")
df3.show()

+--------+----------+-----------+-------+-----+
|order_id|order_date|customer_id| status|newid|
+--------+----------+-----------+-------+-----+
|       4|1564027200|       8827| CLOSED|    3|
|       2|1406260800|        256|PENDING|    1|
|       1|1374724800|      11599| CLOSED|    0|
+--------+----------+-----------+-------+-----+



df3 = [order_id: int, order_date: bigint ... 3 more fields]


[order_id: int, order_date: bigint ... 3 more fields]

In [60]:
// drop the order_id column
val df4=df3.drop("order_id")

df4 = [order_date: bigint, customer_id: int ... 2 more fields]


[order_date: bigint, customer_id: int ... 2 more fields]

In [62]:
// sort the data on the basis od new_id
val df5=df4.sort("newid")
df5.show

+----------+-----------+--------+-----+
|order_date|customer_id|  status|newid|
+----------+-----------+--------+-----+
|1406260800|        256| PENDING|    1|
|1374724800|      11599|COMPLETE|    2|
|1564027200|       8827|  CLOSED|    3|
+----------+-----------+--------+-----+



df5 = [order_date: bigint, customer_id: int ... 2 more fields]


[order_date: bigint, customer_id: int ... 2 more fields]